In [1]:
# # Del all directories in qualitative results which are empty
# import os
# directory = "qualitative_results"
# for d in os.listdir(directory):
#     try:
#         os.rmdir(os.path.join(directory, d))
#     except:
#         print('ok')

In [1]:
!nvidia-smi

Fri Jul 23 20:23:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.03   Driver Version: 450.119.03   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:00:17.0 Off |                    0 |
| N/A   40C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           On   | 00000000:00:18.0 Off |                    0 |
| N/A   

In [2]:
! pip install tensorflow-gpu==2.3.0 # for multi-GPU training

     |████████████████████████████████| 320.4 MB 27 kB/s s eta 0:00:01
     |████████████████████████████████| 6.0 MB 47.7 MB/s eta 0:00:01
     |████████████████████████████████| 459 kB 76.1 MB/s eta 0:00:01
     |████████████████████████████████| 26.1 MB 53.7 MB/s eta 0:00:01    |██████▎                         | 5.1 MB 53.7 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 219 kB/s  eta 0:00:01
     |████████████████████████████████| 781 kB 48.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 62.2 MB/s eta 0:00:01
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstalled tensorflow-estimator-2.1.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Attempting uninstall: scipy
    Found existing installation: scipy

In [9]:
# !rm -rf qualitative_results
# !rm -rf summary
# !rm -rf models
# !rm -rf logs

In [ ]:
# !python3 model.py
!python3 main.py
# !python3 generate_recommendations.py
# !python3 dataset.py
# !python3 config.py

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


# 

# EXP

In [15]:
from dataset import wynk_sessions_dataset
import numpy as np

In [8]:
dataset = wynk_sessions_dataset()


>>>>> Building vocab... >>>>>

Loading build_vocab_dict...
<<<<< Vocab built... <<<<<


>>>>> Mapping song2info... >>>>>

Loading song2info dict...
<<<<< Mapped song2info... <<<<<



In [78]:
import tensorflow as tf
from tensorflow.keras.layers import Layer
import pdb
from config import *

class customLinear(Layer):
    def __init__(self, in_units, out_units):
        super(customLinear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(in_units, out_units), dtype="float32"),
            trainable=True,)
        
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(        
            initial_value=b_init(shape=(out_units,), dtype="float32"), trainable=True)        

        self.w_regularizer = tf.keras.regularizers.l2(1e-5) 

    def get_regularizer_losses(self):
        '''
        calculates and returns the regularizer losses for weights and/or baises
        '''
        # print('\ncalculating regularizer losses...\n')
        
        # w_regularizer_loss = self.w_regularizer(self.w)
        # b_regularizer_loss = self.b_regularizer(self.b)
        #self.add_loss([w_regularizer_loss, b_regularizer_loss])
        # above written code would not work. can not understand why.

        # self.add_loss([lambda: self.w_regularizer(self.w), lambda: self.b_regularizer(self.b)]) 
        # This method (add_loss) can be used inside a subclassed layer or model's call function, 
        # in which case losses should be a Tensor or list of Tensors.
        # https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer

        return [self.w_regularizer(self.w)]

    def call(self, inputs):        
        return tf.matmul(inputs, self.w) + self.b    
      
class rnn_reco_model(tf.keras.Model):
    def __init__(self, vocab_size):        
        super(rnn_reco_model, self).__init__()        
        self.song_emb = tf.keras.layers.Embedding(vocab_size, 
                                                  SONG_EMB_DIM, 
                                                  mask_zero=True, 
                                                  name="song_embedding_layer")               
        
        if USE_TIME_BUCKETS:
            self.time_bucket_emb = tf.keras.layers.Embedding(TIME_BUCKET_VOCAB_SIZE, 
                                                             TIME_BUCKET_EMB_DIM, 
                                                             mask_zero=True, 
                                                             name="time_bucket_embedding_layer")
        
        self.lstm = tf.keras.layers.LSTM(LSTM_DIM, return_state=True, name="rnn_layer", return_sequences=True)  
        self.attn = tf.keras.layers.Dense(1)
        self.dense = customLinear(in_units=LSTM_DIM, out_units=vocab_size)
        self.dense.build((LSTM_DIM, ))
          
    
    
    def call(self, song_emb_inp, time_bucket_emb_inp, initial_state=None, training=True):   
        song_emb = self.song_emb(song_emb_inp)                           # (bs, MAX_LEN, SONG_EMB_DIM)
        song_emb_mask = self.song_emb.compute_mask(song_emb_inp)         # (bs, MAX_LEN)

        lstm_inp = song_emb                                              # (bs, MAX_LEN, SONG_EMB_DIM)
        lstm_inp_mask = song_emb_mask                                    # (bs, MAX_LEN)
                
        if (time_bucket_emb_inp is not None) and USE_TIME_BUCKETS:
            time_bucket_emb = self.time_bucket_emb(time_bucket_emb_inp)  # (bs, MAX_LEN, TIME_BUCKET_EMB_DIM)            
            lstm_inp = tf.concat([lstm_inp, time_bucket_emb], axis = -1) # (bs, MAX_LEN, SONG_EMB_DIM+TIME_BUCKET_EMB_DIM)
            
        lstm, state_h, state_c = self.lstm(lstm_inp, mask=song_emb_mask, initial_state=initial_state)
        
        print("\n===inside call===")
        print("lstm.shape: ", lstm.shape)
#         print("state_h.shape: ", state_h.shape)
#         print("state_c.shape: ", state_c.shape)
        
        attn_weights = self.attn(lstm)
        print("attn_weights.shape: ", attn_weights.shape)  
        
        attn_weights_squeezed = tf.squeeze(attn_weights, axis=-1)
        print("attn_weights_squeezed.shape: ", attn_weights_squeezed.shape)
        
        attn_weights_squeezed_masked = tf.math.multiply(attn_weights_squeezed, tf.cast(lstm_inp_mask, tf.float32))
        print("attn_weights_squeezed_masked.shape: ", attn_weights_squeezed_masked.shape)
        
        attn_weights_softmaxed = tf.keras.layers.Softmax(axis=-1)(attn_weights_squeezed_masked)
        print("attn_weights_softmaxed.shape: ", attn_weights_softmaxed.shape)

        attn_weights_softmaxed_repeated = tf.keras.layers.RepeatVector(LSTM_DIM)(attn_weights_softmaxed)
        print("attn_weights_softmaxed_repeated.shape: ", attn_weights_softmaxed_repeated.shape)
            
        attn_weights_softmaxed_repeated_permuted = tf.keras.layers.Permute([2,1])(attn_weights_softmaxed_repeated)
        print("attn_weights_softmaxed_repeated_permuted.shape: ", attn_weights_softmaxed_repeated_permuted.shape)
            
        weighted_lstm = tf.keras.layers.Multiply()([lstm, attn_weights_softmaxed_repeated_permuted])
        print("weighted_lstm.shape: ", weighted_lstm.shape)
            
            
        weighted_sum_lstm = tf.math.reduce_sum(weighted_lstm, axis=1, keepdims=False)
        print(weighted_sum_lstm.shape)
        
        
        if not training:
            logits = self.dense(lstm)
            probs = tf.nn.softmax(logits, axis=-1)        
            return probs, state_h, state_c 
        else:
            return lstm  
        
USE_ATTENTION = True        
model = rnn_reco_model(dataset.vocab_size)
song_inp = tf.constant(np.random.randint(0, 3, (2,10)))
print("song_inp: ", song_inp)
print("song_inp.shape: ", song_inp.shape)

lstm = model(song_emb_inp=song_inp,
          time_bucket_emb_inp=None)

# temp = RepeatVector(256)(a)  # (?,19) becomes (?,256,19)
# temp = Permute([2,1])(temp)  # change from (?,256,19) to (?,19,256)
# output = Multiply()([x,temp])# Apply weight to each of the 256 dim

song_inp:  tf.Tensor(
[[0 0 0 2 2 1 1 2 1 2]
 [2 2 1 2 0 1 1 1 0 0]], shape=(2, 10), dtype=int64)
song_inp.shape:  (2, 10)

===inside call===
lstm.shape:  (2, 10, 1024)
attn_weights.shape:  (2, 10, 1)
attn_weights_squeezed.shape:  (2, 10)
attn_weights_squeezed_masked.shape:  (2, 10)
attn_weights_softmaxed.shape:  (2, 10)
attn_weights_softmaxed_repeated.shape:  (2, 1024, 10)
attn_weights_softmaxed_repeated_permuted.shape:  (2, 10, 1024)
weighted_lstm.shape:  (2, 10, 1024)
(2, 1024)


In [2]:
import pandas as pd
import numpy as np

from model import rnn_reco_model
from dataset import wynk_sessions_dataset
from config import *

# Make dataset
dataset = wynk_sessions_dataset()

# Initialize model (and load the weights)
model = rnn_reco_model(dataset.vocab_size)   

LOAD_MODEL_PATH = "models/models_withtimebuckets_rnn_7_day_data_2021_06_29_200418_1024_64_10/model_00_124999"
if LOAD_MODEL:
#     model.build(input_shape=((None, MAX_LEN), (None, MAX_LEN)))    
#     print(model.summary())
    print("loading")
    model.load_weights(LOAD_MODEL_PATH)
    print(f"Model loaded: {LOAD_MODEL_PATH}")    

INFO:tensorflow:Using MirroredStrategy with devices ('/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7')
INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7'), communication = CollectiveCommunication.AUTO

>>>>> Building vocab... >>>>>

Loading build_vocab_dict...
<<<<< Vocab built... <<<<<


>>>>> Mapping song2info... >>>>>

Loading song2info dict...
<<<<< Mapped song2info... <<<<<

loading
Model loaded: models/models_withtimebuckets_rnn_7_day_data_2021_06_29_200418_1024_64_10/model_00_124999


In [4]:
song_inp = tf.constant(np.random.randint(1, 3, (1,10)))
time_bucket_inp = tf.constant(np.random.randint(1, 3, (1,10)))
print("song_inp.shape: ", song_inp.shape)
print("time_bucket_inp.shape: ", time_bucket_inp.shape)

# q()
_ = model(song_emb_inp=song_inp,
          time_bucket_emb_inp=time_bucket_inp)

song_inp.shape:  (1, 10)
time_bucket_inp.shape:  (1, 10)


In [8]:
# for i in model.layers:
#     print(type(i), i.name, type(i.weights), len(i.weights))

# q()
# x = model.song_emb.weights[0]
x = model.time_bucket_emb.weights[0]

print(type(x), x.shape)

np.sum(x.numpy())
# x


#         print(model.summary())
#         model.load_weights(LOAD_MODEL_PATH)


<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'> (229, 32)


39.466763

In [12]:
'''
# new model
song 83.67507
time -1.1676371

# trainied model
song -199908.17
time 38.30059
'''

'\n# new model\nsong 83.67507\ntime -1.1676371\n\n# trainied model\nsong -199908.17\ntime 38.30059\n'

In [30]:
# test_song_ids_df = pd.read_csv(QUALITATIVE_TEST_DATA_PATH)
# print(test_song_ids_df.head(20))

# test_song_ids = ["srch_unisysinfo_M09050388", "srch_sonymusic_INS171602370", "srch_sonymusic_698050", "srch_hungama_396873"]
# for s in test_song_ids:
#     print(s, dataset.song2info[s])

# srch_unisysinfo_M09050388 TITLE: Dil Diyan Gallan | ALBUM: Tiger Zinda Hai | ARTIST: ,  | FREQUENCY: 217086 | LANG: hindi | YEAR: 2017
# srch_sonymusic_INS171602370 TITLE: Channa Mereya | ALBUM: Ae Dil Hai Mushkil | ARTIST:  | FREQUENCY: 309043 | LANG: hindi | YEAR: 2016
# srch_sonymusic_698050 TITLE: Maa Tujhe Salam | ALBUM: Vande Mataram Maa Tujhe Salaam | ARTIST:  | FREQUENCY: 21259 | LANG: hindi | YEAR: 2006
# srch_hungama_396873 TITLE: Kun Faya Kun | ALBUM: Rockstar | ARTIST: Mohit Chauhan,A R Rahman,Javed Ali | FREQUENCY: 168921 | LANG: hindi | YEAR: 2011

In [4]:
song_inp = dataset.item2idx["srch_hungama_396873"] # Kun Faya Kun
song_inp = dataset.item2idx["srch_unisysinfo_M09050388"] # Dil Diyan Gallan
song_inp = dataset.item2idx["srch_sonymusic_INS171602370"] # Channa Mereya
song_inp = dataset.item2idx["srch_sonymusic_698050"] # Maa Tujhe Salam

def get_next_song_pred(song_inp, time_bucket_inp, num_recommendation_timesteps=1):
    song_inp = np.array([[song_inp]])
    time_bucket_inp = np.array([[time_bucket_inp]])
    
    exclude = np.zeros([1, dataset.vocab_size]) #(1, vocab_size)

    for idx in song_inp[0]:
        exclude[0, idx] = 1 

    initial_state = None #[state_h, state_c]
    
    for t in range(num_recommendation_timesteps):
        probs, state_h, state_c = model(song_emb_inp=song_inp,
                                        time_bucket_emb_inp=time_bucket_inp,
                                        training=False, 
                                        initial_state=initial_state) # probs (1, num_items)    
        
        probs = probs*(1-exclude)             # probs (1, vocab_size)
        pred = tf.math.argmax(probs, axis=-1) # (1, )
        
        exclude[0, int(pred[0])] = 1 #(1, vocab_size)

        inp = tf.expand_dims(pred, axis = -1) # (1, 1)        
        initial_state = [state_h, state_c]
        next_song_pred = int(pred[0])
                        
    print(f"time {time_bucket_inp[0, 0]}: {int(pred[0])}: {probs[0, int(pred[0])].numpy()}")
    return next_song_pred

for t in [int(i) for i in np.linspace(0, 228, num=50)]:
    print("================")
    next_song_pred = get_next_song_pred(song_inp=song_inp, time_bucket_inp=t)
    print(dataset.song2info[dataset.idx2item[next_song_pred]])



time 0: 17997: 0.01167581882327795
TITLE: Bharat Humko Jaan Se Pyara Hai | ALBUM: Roja | ARTIST: Hariharan,A R Rahman | FREQUENCY: 2980 | LANG: hindi | YEAR: 1994
time 4: 5611: 0.00964123010635376
TITLE: Mitwa | ALBUM: Lagaan | ARTIST: Alka Yagnik,Udit Narayan,Srinivas,Sukhwinder Singh,A R Rahman | FREQUENCY: 13747 | LANG: hindi | YEAR: 2001
time 9: 5611: 0.00972586777061224
TITLE: Mitwa | ALBUM: Lagaan | ARTIST: Alka Yagnik,Udit Narayan,Srinivas,Sukhwinder Singh,A R Rahman | FREQUENCY: 13747 | LANG: hindi | YEAR: 2001
time 13: 5611: 0.01098481472581625
TITLE: Mitwa | ALBUM: Lagaan | ARTIST: Alka Yagnik,Udit Narayan,Srinivas,Sukhwinder Singh,A R Rahman | FREQUENCY: 13747 | LANG: hindi | YEAR: 2001
time 18: 5611: 0.010092789307236671
TITLE: Mitwa | ALBUM: Lagaan | ARTIST: Alka Yagnik,Udit Narayan,Srinivas,Sukhwinder Singh,A R Rahman | FREQUENCY: 13747 | LANG: hindi | YEAR: 2001
time 23: 5611: 0.009190144948661327
TITLE: Mitwa | ALBUM: Lagaan | ARTIST: Alka Yagnik,Udit Narayan,Srinivas,S